# Gaussian Elimination

### Naive Gaussian Elimination

* 消除第 i 列的方法

    要求 $a_{ii} \neq 0$，对 $j = i+1,...,n$ 行，该行减去 $\frac{a_{ji}}{a_{ii}} \cdot 第 i 行$ 

* 时间复杂度为 $O(n^3)$

* back-substitution 时间复杂度为 $O(n^2)$

### LU 分解

对方阵进行高斯消元的过程，即可获得该方阵的 $LU$ 分解

* 已知 $LU$ 分解的 back-substitutio

$$A x = b = LUx = Lc$$ 

先通过回代 $Lc = b$ 解出 $c$，再回代 $Ux = c$ 解出 $x$

### 向量和矩阵的模

* 向量的模 $||x||$
    1. $||x|| \geq 0$， 当且仅当 $||x||$ 为零向量时等于 $0$
    2. 对标量 $\alpha$ 满足 $||\alpha x|| = |\alpha| \cdot ||x||$
    3. $||x+y||\leq ||x|| + ||y||$


* 矩阵的模 $||A||$
    1. $||A|| \geq 0$， 当且仅当 $||A||$ 为零矩阵时等于 $0$
    2. 对标量 $\alpha$ 满足 $||\alpha A|| = |\alpha| \cdot ||A||$
    3. $||A+B||\leq ||A|| + ||B||$

* 方阵的 **operator norm**\
    遍历所有非零向量 $|x|$，可定义 operator 模\
    $||A|| = max \frac{||Ax||}{||x||}$

### 前向误差与后向误差

对线性方程组 $Ax=b$ 的求解问题，近似解为 $x_a$，则残差（residual）为 $r = b - A x_a$

* 后向误差\
    $||r||_{\infty} = ||b - A x_a||_{\infty}$

* 相对后向误差\
    $\frac{||r||_{\infty}}{||b||_{\infty}}$

* 前向误差\
    $||x-x_a||_{\infty}$

* 相对前向误差\
    $\frac{||x-x_a||_{\infty}}{||x||_{\infty}}$

* 误差放大因子\
    error magnification factor $= \frac{相对前向误差}{相对后向误差} = \frac{\frac{||x-x_a||_{\infty}}{||x||_{\infty}}}{\frac{||r||_{\infty}}{||b||_{\infty}}}$

* 方阵 $A$ 的 $condition\ number\ cond(A)$\
    $cond(A)$ 定义为对所有可能的 $b$, 求解 $Ax=b$ 的误差放大因子的最大值

* $n \times n$ 方阵 $A$ 的模\
    $||A||_{\infty}$: 对每一行，求该行所有元素的绝对值的和，$n$ 个和中的最大值为方阵 $A$ 的模

    $||A||_{1}$: 对每一列，求该列所有元素的绝对值的和，$n$ 个和中的最大值为方阵 $A$ 的模

* 定理
    $cond(A) = ||A|| \cdot ||A^{-1}||$\
    （由 operator norm 的定义可证明）
    

#### Example

希尔伯特矩阵 $H, H_{ij} = \frac{1}{i + j - 1}$, 有着很大的 condition number。对 $n = 6\ 和\ 10$，求解 $H x = b$, 其中 $b = H \cdot [1, ..., 1]^T$

In [1]:
import numpy as np

def hibert_matrix(n):
    mat = [[1.0 / (i + j - 1) for j in range(1, n + 1)] for i in range(1, n + 1)]
    return np.matrix(mat)

def norm(A, n):
    sums = []
    for j in range(0, n):
        sum = 0
        for i in range(0, n):
            # print("i = {}, j = {}".format(i, j))
            sum += abs(A[(i, j)])
        sums.append(sum)
    max = sums[0]
    for j in range(0, n):
        if sums[j] > max:
            max = sums[j]
    return max

def cond(A, n):
    inv_A = np.linalg.inv(A)
    norm_A = norm(A, n)
    norm_inv_A = norm(inv_A, n)
    cond_A = norm_A * norm_inv_A
    return cond_A

A = hibert_matrix(6)
print("cond(A, 6) = {:e}".format(cond(A, 6)))
ones = np.matrix([[1.0] for i in range(0, 6)])
b = A @ ones
inv_A = np.linalg.inv(A)
x_a = inv_A @ b
print("x_a = ")
for j in x_a:
    print("{:.16f}".format(j[(0,0)]))

A = hibert_matrix(10)
print("cond(A, 10) = {:e}".format(cond(A, 10)))
ones = np.matrix([[1.0] for i in range(0, 10)])
b = A @ ones
inv_A = np.linalg.inv(A)
x_a = inv_A @ b
print("x_a = ")
for j in x_a:
    print("{:.16f}".format(j[(0,0)]))


cond(A, 6) = 2.907028e+07
x_a = 
0.9999999994633981
1.0000000011641532
0.9999999993015081
1.0000000009313226
0.9999999995343387
1.0000000000582077
cond(A, 10) = 3.535372e+13
x_a = 
0.9997739568352699
1.0030155181884766
0.9954032897949219
1.0023498535156250
0.9975585937500000
1.0000000000000000
1.0009765625000000
1.0000000000000000
1.0000000000000000
0.9998779296875000


### PA = LU

#### 置换矩阵 (permutation matrices)

将单位矩阵的行之间进行置换之后的矩阵

#### partial pivoting

对每一列进行消元时，通过行置换选择绝对值最大的主元

解决两个问题：

 1. 消除主元为 0 的情况
 2. 消除主元太小导致的数值误差

#### PA = LU 分解

* 对每一列进行消元前通过 partial pivoting 进行主元置换，置换过程记录在置换矩阵中

* (i, j) 元素的消除系数记录在原来的位置上，并参与之后的行置换，但不参与之后的加减乘除

* 分解后得到 P, L, U 三个矩阵

#### 回代过程

$P A x = P b = L U x = L c$

易得 $c = U x$

易得 $x$